In [ ]:
!pip install poloniex
!pip install git+https://github.com/preslavrachev/pandas-ta.git
import pandas as pd
import poloniex
import time
from pandasta import indicators as pdt

# Fetch the Data
For training and testing data, we will be using the [Poloniex public API](https://poloniex.com/support/api/).

In [24]:
end = int(time.time())
period = 900
start = end - 30000 * period
data = poloniex.Poloniex().returnChartData('USDT_BTC',
                                           period=period,
                                           start=start,
                                           end=end)

In [25]:
df = pd.DataFrame(data)
df['time'] = df['date']
df['close'] = df['close'].astype('float32')

## Indicators / Features
Using the Pandas TA conventions, we will request a few indicators, based on different tiem windows. In our current case, this will be the fast-moving part (K) of the [Stochastic Oscillator](https://en.wikipedia.org/wiki/Stochastic_oscillator) index.

In [26]:
cluster_features = ['stochk_10',
                    'stochk_20',
                    'stochk_50',
                    'stochk_100',
                    'stochk_200',
                    'stochk_300',
                    'stochk_400',
                    'stochk_800']

In [27]:
df = pdt.TaDataFrame(df, indicators=cluster_features)
df.tail()

,close,date,high,low,open,quoteVolume,volume,weightedAverage,stochk_10,stochk_20,stochk_50,stochk_100,stochk_200,stochk_300,stochk_400,stochk_800
time,,,,,,,,,,,,,,,,
2018-08-24 04:45:00,6503.701660,1535085900,6508.87532999,6489.44726019,6489.44726028,1.98271505,12887.90312908,6500.12876489,0.476242,0.423588,0.464888,0.516637,0.478149,0.504108,0.495668,0.508473
2018-08-24 05:00:00,6504.710449,1535086800,6504.80438,6502.71236438,6503.70178829,0.15978222,1039.20409648,6503.87819423,0.439199,0.483465,0.479236,0.516686,0.477685,0.505420,0.496100,0.508474
2018-08-24 05:15:00,6504.074707,1535087700,6506.71172,6498.76663998,6506.71172,2.23403516,14519.82491677,6499.37170942,0.587614,0.520485,0.486803,0.517181,0.475663,0.505140,0.496932,0.508206
2018-08-24 05:30:00,6506.000000,1535088600,6506,6498.76664343,6498.76664343,1.80887075,11766.8782218,6505.09618877,0.615846,0.523841,0.492674,0.520786,0.477447,0.504944,0.497220,0.508135
2018-08-24 05:45:00,6514.844238,1535089500,6514.84437,6506,6506,0.1544792,1006.29751789,6514.12952616,0.741059,0.622291,0.490733,0.524215,0.477059,0.505764,0.497056,0.507980


More to come soon...